In [1]:
import random, time, threading, os, glob
from random import randint
import xml.etree.ElementTree as ET
from xml.dom import minidom
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr
from pyspark.sql.types import *

host = 'spark-test1'
checkpoint = 'hdfs://spark-test1:9000/checkpoint/raw/transactions'
xml_directory = 'hdfs://spark-test1:9000/in/transactions'

# Set the location of the Delta Lake and Kafka packages
kafka_package = "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.0"  # Replace with the correct Spark version

# Initialize Spark Session for Kafka
spark = SparkSession \
    .builder \
    .appName("XMLToKafkaProducer") \
    .master(f"spark://{host}:7077") \
    .config("spark.jars.packages", f"{kafka_package}") \
    .config("spark.sql.streaming.checkpointLocation", checkpoint) \
    .config("spark.cores.max", "1") \
    .getOrCreate()

# Kafka Configuration
kafka_server = f"{host}:9092"
topic_name = "test-topic"

schema = StructType([
    StructField("path", StringType(), False),
    StructField("modificationTime", TimestampType(), False),
    StructField("length", LongType(), False),
    StructField("content", BinaryType(), True)
])

# Read stream from directory
df = spark.readStream.format("binaryFile") \
    .schema(schema) \
    .load(xml_directory)



# Write the stream to Kafka
query = df.selectExpr("path as key", "to_json(struct(*)) AS value").writeStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_server) \
    .option("topic", topic_name) \
    .start()

# query = df \
#     .writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()
    
query.awaitTermination()

:: loading settings :: url = jar:file:/home/spark/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/spark/.ivy2/cache
The jars for the packages stored in: /home/spark/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2dd06a9f-d59d-4ce0-a071-cbd6aab7faaf;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.5.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.5.0 in central
	found org.apache.kafka#kafka-clients;3.4.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.10.3 in central
	found org.slf4j#slf4j-api;2.0.7 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.4 in central
	found org.apache.hadoop#hadoop-client-api;3.3.4 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
:: resolution report :: resolve 302ms :: artifacts dl 8ms
	:: mo

Py4JError: An error occurred while calling o52.awaitTermination